## **Subqueries and Window Functions in SQL**

###  `Setting Up the Database`
##### We'll create a sample database and tables to use in our examples

In [ ]:
-- Active: 1723560744721@@127.0.0.1@3306@sakila
-- CREATE DATABASE SalesDB;
-- USE SalesDB;
-- Drop Database SalesDB;

CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    Department VARCHAR(50),
    Salary DECIMAL(10, 2),
    HireDate DATE
);

CREATE TABLE Sales (
    SaleID INT PRIMARY KEY,
    EmployeeID INT,
    SaleAmount DECIMAL(10, 2),
    SaleDate DATE,
    FOREIGN KEY (EmployeeID) REFERENCES Employees(EmployeeID)
);


In [ ]:
-- Inserting sample data
INSERT INTO Employees VALUES
(1, 'John', 'Doe', 'Sales', 60000, '2015-03-15'),
(2, 'Jane', 'Smith', 'Sales', 70000, '2016-05-22'),
(3, 'Mike', 'Johnson', 'IT', 90000, '2018-08-01'),
(4, 'Emily', 'Davis', 'HR', 75000, '2017-11-13');

INSERT INTO Sales VALUES
(101, 1, 5000, '2024-07-01'),
(102, 1, 7000, '2024-07-15'),
(103, 2, 6000, '2024-07-05'),
(104, 2, 8000, '2024-07-20'),
(105, 1, 9000, '2024-07-25');

## **Subqueries**

## What is a `Subquery`?

#### A subquery is a query nested inside another query. It can be used to return data that will be used by the main query as a condition to further restrict the data to be retrieved. Subqueries are often used in WHERE, SELECT, FROM, and HAVING clauses.

## `Types of Subqueries`

#### `Single-Row Subquery`: Returns a single row.
#### `Multi-Row Subquery`: Returns multiple rows.
#### `Correlated Subquery`: The subquery depends on the outer query for its values.
#### `Nested Subquery`: A subquery within another subquery.


## **Examples of Subqueries**

### **Single-Row Subquery**

In [ ]:
select * from employees

In [ ]:
-- Let's find the employee with the highest salary:


SELECT  MAX(Salary) as max_salary FROM Employees

SELECT * FROM Employees
WHERE Salary = (SELECT MAX(Salary) FROM Employees);


-- Explanation:

-- The subquery (SELECT MAX(Salary) FROM Employees) returns the maximum salary.
-- The main query retrieves the employee details with that salary

### **Multi-Row Subquery**

In [ ]:
-- find employees whose salary is higher than the average salary in the company
-- what is the average salary in the company?
--- which employees have sal > avg sal.

SELECT AVG(Salary) FROM Employees 

SELECT * FROM Employees
WHERE Salary > (SELECT AVG(Salary) FROM Employees);


-- Explanation:

-- The subquery (SELECT AVG(Salary) FROM Employees) calculates the average salary.
-- The main query selects employees whose salary is greater than the average

### **Nested Subquery**

In [ ]:
-- find the employees who have made a sale on the most recent sale date:

SELECT * FROM Employees
WHERE EmployeeID IN (
    SELECT EmployeeID FROM Sales
    WHERE SaleDate = (SELECT MAX(SaleDate) FROM Sales)
);

-- Explanation:

-- The innermost subquery finds the most recent sale date.
-- The middle subquery retrieves the EmployeeID of those who made a sale on that date.
-- The outer query retrieves the employee details.


In [ ]:
-- find the details of the most recent sale made by each employee

--101

SELECT EmployeeID, SaleID, SaleAmount, SaleDate
FROM Sales
WHERE SaleDate = (
    SELECT MAX(SaleDate)
    FROM Sales s2
    WHERE s2.EmployeeID = Sales.EmployeeID
);

-- Explanation:

-- The subquery depends on the outer query (e.EmployeeID).
-- It checks for each employee if there's a sale they made that exceeds their average sale amount.


## **Correlated Subquery**

In [ ]:
-- find employees who have made sales that are greater than the average sale amount for each employee:

--what is the avg sales amount for each employee?

SELECT AVG(SaleAmount) FROM Sales s 


SELECT EmployeeID, FirstName, LastName FROM Employees e
WHERE EXISTS (
    SELECT 1 FROM Sales s
    WHERE s.EmployeeID = e.EmployeeID
    AND s.SaleAmount > (SELECT AVG(SaleAmount) FROM Sales s WHERE s.EmployeeID = e.EmployeeID)
);


#101--->saleamt--> greater than avg (salesamt)

-- Explanation:

-- Main Query:

-- The outer query selects EmployeeID, FirstName, and LastName from the Employees table.
-- Correlated Subquery:

-- The subquery inside the WHERE EXISTS clause is executed for each row of the outer query.
-- s.EmployeeID = e.EmployeeID: This part of the subquery links the subquery to the current row of the outer query (correlation). It ensures we are checking sales made by the same employee.
-- The subquery checks if there is any sale amount (SaleAmount) for that employee which is greater than their average sale amount.
-- EXISTS returns TRUE if the subquery finds at least one row matching the condition.
-- Outcome:

-- The query returns employees who have at least one sale above their average sale amount.

## **Examples of Window Functions**

### **`ROW_NUMBER()`**

In [ ]:
-- Let's assign a unique row number to each employee in the Sales department based on their salary, ordered from highest to lowest

SELECT EmployeeID, FirstName, LastName, Salary,Department,
       ROW_NUMBER() OVER (PARTITION BY Department ORDER BY Salary DESC) as RowNum
FROM Employees;
--WHERE Department = 'Sales';


-- Explanation:

-- PARTITION BY Department divides the data by department.
-- ORDER BY Salary DESC orders the employees by salary in descending order within the Sales department.
-- ROW_NUMBER() assigns a unique number to each row.


### **`RANK()`**

In [ ]:
-- Let's rank the employees based on their salary:

SELECT EmployeeID, FirstName, LastName, Salary,Department,
       RANK() OVER (PARTITION BY Department ORDER BY Salary DESC) as SalaryRank
FROM Employees;


-- Explanation:

-- RANK() assigns a rank to each employee based on their salary.
-- Employees with the same salary receive the same rank, and the next rank is skipped.


### **`DENSE_RANK()`**

In [ ]:
-- Let's use DENSE_RANK() to avoid gaps in ranking:

SELECT EmployeeID, FirstName, LastName, Salary,
       DENSE_RANK() OVER (ORDER BY Salary DESC) as SalaryDenseRank
FROM Employees;

-- Explanation:

-- DENSE_RANK() is similar to RANK(), but it doesn’t skip ranks when there are ties

### **`SUM()` as a Window Function**

In [ ]:
--calculate the cumulative sales made by each employee

SELECT EmployeeID, SaleID, SaleAmount,
       SUM(SaleAmount) OVER (PARTITION BY EmployeeID ORDER BY SaleDate) as CumulativeSales
FROM Sales;

--calculate the avg salary per department.
-- HW
-- select EmployeeID,FirstName,LastName,Department,avg_sal
-- from employees IN
-- (select *, 
--     avg(Salary) over (partition by Department) as avg_sal
--     from Employees)



-- Explanation:

-- PARTITION BY EmployeeID calculates the cumulative sum for each employee.
-- ORDER BY SaleDate ensures that the cumulative sum is calculated in the order of the sales.

### **`LEAD() and LAG()`**

In [ ]:
--use LEAD() to find the difference in salary between the current employee and the next highest paid employee:

SELECT EmployeeID, FirstName, LastName, Salary,
       LEAD(Salary, 1) OVER (ORDER BY Salary DESC) as NextHigherSalary,
       Salary - LEAD(Salary, 1) OVER (ORDER BY Salary DESC) as SalaryDifference
FROM Employees;


-- SELECT EmployeeID, FirstName, LastName, Salary,
--        LEAD(Salary, 1) OVER (ORDER BY Salary DESC) as NextHigherSalary,
--        LAG(Salary,1) OVER (ORDER BY Salary DESC) as SalaryDifference
-- FROM Employees;

-- Explanation:

-- LEAD(Salary, 1) gets the salary of the next row (i.e., the next highest paid employee).
-- Salary - LEAD(Salary, 1) calculates the difference between the current employee's salary and the next

### **`NTILE()`**

In [ ]:
-- divide employees into quartiles based on their salary:

SELECT EmployeeID, FirstName, LastName, Salary,
       NTILE(4) OVER (ORDER BY Salary DESC) as SalaryQuartile
FROM Employees;

-- Explanation:

-- NTILE(4) divides the employees into four quartiles based on their salary.
-- ORDER BY Salary DESC ensures the highest salaries are in the first quartile.